# PCA Plots

## General settings and loading files

In [1]:
% matplotlib inline
% load_ext autoreload
% autoreload 2

% run general_settings.py
# ^ Takes ~20s to run if it finds the dumpfiles
# ^ WARNING: It will hog ~2.02 Gb RAM


[12:52:55] 'panels' dict
[12:52:55] 'galanter', 'present', 'missing' dataframes
[12:52:55] 'panel_labels'
[12:52:55] 'panel_names' dict
[12:52:55] 'genome' dataframe
[12:53:00] 'lat' dataframe
[12:53:12] 'control_genotypes' huge datagrame
[12:53:12] 'control_rsIDs' dict to filter it ^
[12:53:12] 'cp_factors' list
[12:53:12] 'control_names' dict
[12:53:12] 'df_1000G_samples'
[12:53:12] 'df_1000G_SNPs'
[12:53:12] 'df_1000G_genotypes'
[12:53:12] 'df_1000G_populations'
[12:53:12] 'df_1000G_genotypes_alleles'

/home/juan/repos/tesina/settings/genome.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  centromers["chromosome"] = [i.replace("CEN", "") for i in centromers.index]
/home/juan/anaconda3/lib/python3.5/site-packages/pandas/io/parsers.py:1572: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())



[12:53:13] 'mafs' dataframe
=> You should check your RAM! <=


/home/juan/anaconda3/lib/python3.5/site-packages/pandas/io/parsers.py:1574: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


## Create the datasets
Lists of sample IDs of chosen populations to later filter 1000 Genomes data.

In [ ]:
from collections import OrderedDict
from os.path import join

DATASETS_DIR = "/home/juan/tesina/dataset_dumps/"


def sample_IDs_from_popcode(pop_dict):  
    populations = []
    for label, pop_list in pop_dict.items():
        mask = df_1000G_samples.population.isin(pop_list)
        populations.append(df_1000G_samples[mask])

    return pd.concat(populations).index

dataset_names = {
    "L": "Latinos",
    "LE": "Latinos, Europeos",
    "LEA": "Latinos, Europeos, Africanos",
    "LEAC": "Latinos, Europeos, Africanos, Chinos",
    "LEACI": "Latinos, Europeos, Africanos, Chinos, Indios",
}

# I could export some of these to .yaml ?

latinos = {'latinos': ['PEL', 'MXL', 'CLM', 'PUR']}
e = {'europeans': ['IBS', 'GBR']}
ea = {**e, **{'africans': ['YRI', 'LWK']}}
eac = {**ea, **{'east_asians': ['CHB', 'CHS']}}
eaci = {**eac, **{'south_asians': ['GIH']}}

samples = OrderedDict()
samples["L"] = latinos
samples["LE"] = {**e, **latinos}
samples["LEA"] = {**ea, **latinos}
samples["LEAC"] = {**eac, **latinos}
samples["LEACI"] = {**eaci, **latinos}

# Transform population codes into 1000 Genome sample IDs
# And write the sample IDs list to a file with the dataset name
for dataset_label, pop_dict in samples.items():
    samples[dataset_label] = sample_IDs_from_popcode(pop_dict)
    filename = join(DATASETS_DIR, dataset_label) + ".samples"
    samples[dataset_label].values.tofile(filename, sep="\n", format="%s")

## Plot settings

In [ ]:
import yaml

plot_colors = yaml.load(open("./settings/plot_colors.yml", "r"))
plot_markers = yaml.load(open("./settings/plot_markers.yml", "r"))

In [ ]:
rsIDs_per_panel = OrderedDict()

for panel_label, panel in panels.items():
    name = panel_names[panel_label]
    rsIDs_per_panel[name] = panel.index

# I already have control_rsIDs !
# for factor in cp_factors:
#     name = control_names[factor]
#     rsIDs_per_panel[name] = control_genotypes[factor].columns

## PCA plots

In [ ]:
from plotters.pca import PCAPlotter

pca_plotter = PCAPlotter()

for dataset_label, sample_ids in samples.items():
    # GAL Panels
    dataset_genotypes = df_1000G_genotypes.loc[sample_ids, :]  # ":" gets all rows, all the SNPs

    fig_title = "Dataset: {}".format(dataset_names[dataset_label])
    pca_plotter.plot(fig_title, rsIDs_per_panel, dataset_genotypes, df_1000G_samples,
                     plot_markers, plot_colors)
    
    # Control Panels
    dataset_genotypes = control_genotypes.loc[sample_ids, :]  # ":" gets all rows, all the SNPs
    pca_plotter.plot()
    
# Save 'em directly to disk, not manually you chipmunk